In [4]:
import shutil
from pathlib import Path

bids_preprocessed = Path('/Volumes/Vol6/YouthPTSD/BIDS_Processed')
scratch_preproc = Path('/scratch/jdrussell3/bidspreproc')
subjdirs = (subjdir for subjdir in scratch_preproc.iterdir() if subjdir.is_dir())

for subjdir in sorted(subjdirs):
    sesdirs = (sesdir for sesdir in subjdir.iterdir() if sesdir.is_dir())
    for sesdir in sesdirs:
        dti_dir = sesdir / 'dti'
        orig_dir = dti_dir / 'original'
        preproc_dir = dti_dir / 'preprocessed'
        preproc_dir_new = bids_preprocessed / subjdir.name / sesdir.name / 'dwi' / 'preprocessed'
        print(preproc_dir_new)
        shutil.copytree(preproc_dir, preproc_dir_new)
        
        
        

/Volumes/Vol6/YouthPTSD/BIDS_Processed/sub-001/ses-01/dwi/preprocessed
/Volumes/Vol6/YouthPTSD/BIDS_Processed/sub-003/ses-01/dwi/preprocessed
/Volumes/Vol6/YouthPTSD/BIDS_Processed/sub-004/ses-01/dwi/preprocessed
/Volumes/Vol6/YouthPTSD/BIDS_Processed/sub-005/ses-01/dwi/preprocessed
/Volumes/Vol6/YouthPTSD/BIDS_Processed/sub-006/ses-01/dwi/preprocessed
/Volumes/Vol6/YouthPTSD/BIDS_Processed/sub-009/ses-01/dwi/preprocessed
/Volumes/Vol6/YouthPTSD/BIDS_Processed/sub-011/ses-01/dwi/preprocessed
/Volumes/Vol6/YouthPTSD/BIDS_Processed/sub-011/ses-02/dwi/preprocessed
/Volumes/Vol6/YouthPTSD/BIDS_Processed/sub-012/ses-01/dwi/preprocessed
/Volumes/Vol6/YouthPTSD/BIDS_Processed/sub-013/ses-01/dwi/preprocessed
/Volumes/Vol6/YouthPTSD/BIDS_Processed/sub-014/ses-01/dwi/preprocessed
/Volumes/Vol6/YouthPTSD/BIDS_Processed/sub-019/ses-01/dwi/preprocessed
/Volumes/Vol6/YouthPTSD/BIDS_Processed/sub-019/ses-02/dwi/preprocessed
/Volumes/Vol6/YouthPTSD/BIDS_Processed/sub-020/ses-01/dwi/preprocessed
/Volum

In [1]:
import string, os, sys, shutil
from pathlib import Path

def dwi_corr(sourcedwi, sourcebvec, sourcebval, sourcefmap_rads, sourcefmap_mag, preprocsubj_dir, \
             procsubj_dir, proceddy_options, dwelltime, totalreadouttime):
    
    ##################################################################################
    #----Creating Directory Structures, Copying Files, and Initializing Variables----#
    ##################################################################################
    
    #1. Get sub-XXX_ses-YY string
    prefix = "_".join(Path(sourcedwi).name.split('_')[0:2])
    print(prefix)
                
    #2. Create directory structure
    origdir = Path(preprocsubj_dir, 'original')
    origdir.mkdir(parents=True, exist_ok=True)
    preprocdir = Path(preprocsubj_dir, 'preprocessed')
    preprocdir.mkdir(parents=True, exist_ok=True)
    
    #3. Copy source files to 'original' directory
    inputdwi = origdir / (prefix + "_dwi.nii")
    inputbvec = origdir / (prefix + "_dwi.bvec")
    inputbval = origdir / (prefix + "_dwi.bval")
    inputfmap_rads = origdir / (prefix + "_fmap_rads.nii")
    inputfmap_mag = origdir / (prefix + "_fmap_mag.nii")
    shutil.copyfile(sourcedwi, inputdwi)
    shutil.copyfile(sourcebvec, inputbvec)
    shutil.copyfile(sourcebval, inputbval)
    shutil.copyfile(sourcefmap_rads, inputfmap_rads)
    shutil.copyfile(sourcefmap_mag, inputfmap_mag)
       
    ############################################
    #----Removing Gibbs Rings and Denoising----#
    ############################################
    
    #1. Convert to MIF format
    dwi_raw = preprocdir/(prefix + "_dwi.mif")
    !mrconvert -force -fslgrad $inputbvec $inputbval $inputdwi $dwi_raw
    
    #2. Denoise #https://www.ncbi.nlm.nih.gov/pubmed/27523449
    dwi_den = preprocdir/(prefix + "_dwi_den.mif")
    !dwidenoise -force $dwi_raw $dwi_den
    
    #3. Remove Gibbs rings #https://onlinelibrary.wiley.com/doi/full/10.1002/mrm.26054
    dwi_den_deg = preprocdir/(prefix + "_dwi_den_deg.mif")
    !mrdegibbs -force $dwi_den $dwi_den_deg

    #################################
    #----Eddy Current Correction----#
    #################################
    #https://www.sciencedirect.com/science/article/pii/S1053811915009209
    #dwifslpreproc: https://mrtrix.readthedocs.io/en/dev/dwi_preprocessing/dwifslpreproc.html
        
    #1. Eddy Current Correction # - MRtrix automatically includes rotated bvecs in output file
    dwi_den_deg_preproc = preprocdir/(prefix + "_dwi_den_deg_preproc.nii.gz")
    outputdir_eddyqc = str(preprocdir/'eddy') #dwifslpreproc kicks back an error if this isn't a string (e.g., PosixPath)
    bvec_posteddy = preprocdir/(prefix + '_bvec_posteddy.bvec')
    bval_posteddy = preprocdir/(prefix + '_bval_posteddy.bval')
    !dwifslpreproc -force -info $dwi_den_deg $dwi_den_deg_preproc -pe_dir AP -rpe_none -scratch '/tmp' \
        -eddy_options $eddy_options -eddyqc_all $outputdir_eddyqc -readout_time $totalreadouttime \
        -export_grad_fsl $bvec_posteddy $bval_posteddy
    
    ###################################
    #----EPI Distortion Correction----#
    ###################################
    #https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4819327/pdf/nihms767022.pdf

    #----Prepare Fieldmap and Magnitude Images----#
    
    #1. Copy the fieldmap (phase difference in radians) and magnitude images to temp directory
    native_fmap_ph = preprocdir/(prefix + "_native_fmap_ph.nii")
    native_fmap_mag = preprocdir/(prefix + "_native_fmap_mag.nii")
    !fslmaths $inputfmap_rads $native_fmap_ph
    !fslmaths $inputfmap_mag $native_fmap_mag
    
    #2. Create a mean b0 image, then skull strip it and save a binary mask file
    native_b0 = preprocdir/(prefix + "_native_b0.nii")
    native_mnb0 = preprocdir/(prefix + "_native_b0_brain.nii")
    native_mnb0_brain = preprocdir/(prefix + "_native_mnb0_brain.nii.gz")
    native_mnb0_brain_mask = preprocdir/(prefix + "_native_mnb0_brain_mask.nii.gz") 
    !fslroi $inputdwi $native_b0 0 7
    !fslmaths $native_b0 -Tmean $native_mnb0
    !bet2 $native_mnb0 $native_mnb0_brain -m
    
    #3. Skull strip the magnitude image and save a binary mask file
    native_fmap_mag_brain = preprocdir/(prefix + "_native_fmap_mag_brain.nii.gz")
    native_fmap_mag_brain_mask = preprocdir/(prefix + "_native_fmap_mag_brain_mask.nii.gz")
    !bet2 $native_fmap_mag $native_fmap_mag_brain -m -f 0.3

    #4. Mask the fieldmap using the binary mask of the magnitude image
    native_fmap_ph_brain = preprocdir/(prefix + "_native_fmap_ph_brain.nii.gz")
    !fslmaths $native_fmap_ph -mas $native_fmap_mag_brain_mask $native_fmap_ph_brain

    #5. Smooth the fieldmap
    native_fmap_ph_brain_s4 = preprocdir/(prefix + "_native_fmap_ph_brain_s4.nii.gz")
    !fugue --verbose --loadfmap=$native_fmap_ph_brain -s 4 --savefmap=$native_fmap_ph_brain_s4
    
    #6. Warp the magnitude image
    native_fmap_mag_brain_warp = preprocdir/(prefix + "_native_fmap_mag_brain_warp.nii.gz")
    !fugue -v -i $native_fmap_mag_brain --unwarpdir=y --dwell=$dwelltime --loadfmap=$native_fmap_ph_brain_s4 -w $native_fmap_mag_brain_warp
    
    #----Register Fieldmap to DTI and Apply It----#
    
    #7. Register the warped magnitude image to the mean B0 image and save the affine matrix
    fmap_mag_brain_warp_reg_2_mnb0_brain = preprocdir/(prefix + "_fmap_mag_brain_warp_reg_2_mnb0_brain.nii.gz")
    fieldmap_2_mnb0_brain_mat = preprocdir/(prefix + "_fieldmap_2_mnb0_brain.mat")
    !flirt -in $native_fmap_mag_brain_warp -ref $native_mnb0_brain -out $fmap_mag_brain_warp_reg_2_mnb0_brain -omat $fieldmap_2_mnb0_brain_mat

    #8. Use the warped_magnitude-2-meanB0 matrix to register the smoothed fieldmap to the full DTI set.
    fmap_ph_brain_s4_reg_2_mnb0_brain = preprocdir/(prefix + "_fmap_ph_brain_s4_reg_2_mnb0_brain.nii.gz")
    !flirt -in $native_fmap_ph_brain_s4  -ref $native_mnb0_brain -applyxfm -init $fieldmap_2_mnb0_brain_mat -out $fmap_ph_brain_s4_reg_2_mnb0_brain
    
    #9. Warp the DTI volumes using the newly registered fieldmap.
    dwi_den_deg_preproc_warp_niigz = preprocdir/(prefix + "_dwi_den_deg_preproc_warp.nii.gz")
    !fugue -v -i $dwi_den_deg_preproc --icorr --unwarpdir=y --dwell=$dwelltime --loadfmap=$fmap_ph_brain_s4_reg_2_mnb0_brain -u $dwi_den_deg_preproc_warp_niigz
    
    ###############################################
    #----Bias Field (B1) Distortion Correction----#
    ###############################################
    #https://www.ncbi.nlm.nih.gov/pubmed/20378467
    
    #1. Convert back to .MIF
    dwi_den_deg_preproc_warp_mif = preprocdir/(prefix + "_dwi_den_deg_preproc_warp.mif")
    !mrconvert -force -fslgrad $bvec_posteddy $bval_posteddy $dwi_den_deg_preproc_warp_niigz $dwi_den_deg_preproc_warp_mif 

    #2. Bias correction
    dwi_den_deg_preproc_warp_unb = preprocdir/(prefix + "_dwi_den_deg_preproc_warp_unb.mif")
    !dwibiascorrect -force ants $dwi_den_deg_preproc_warp_mif $dwi_den_deg_preproc_warp_unb -scratch '/tmp'

    ######################################
    #----Upsampling and Mask Creation----#
    ######################################
    
    #1. Regridding to 1.5mm isomorphic voxels #Suggested on 3tissue.github.io/doc/single-subject.html
    dwi_den_deg_preproc_warp_unb_ups = preprocdir/(prefix + "_dwi_den_deg_preproc_warp_unb_ups.mif")
    !mrgrid -force $dwi_den_deg_preproc_warp_unb regrid $dwi_den_deg_preproc_warp_unb_ups -voxel 1.5
    
    #2. Mask generation
    
    ##a. Extract b0s
    dwi_den_deg_preproc_warp_unb_b0s = preprocdir/(prefix + "_dwi_den_deg_preproc_warp_unb_b0s.mif")
    !dwiextract -force -bzero $dwi_den_deg_preproc_warp_unb $dwi_den_deg_preproc_warp_unb_b0s
    
    ##b. Compute mean b0
    dwi_den_deg_preproc_warp_unb_meanb0 = preprocdir/(prefix + "_dwi_den_deg_preproc_warp_unb_meanb0.mif")
    !mrmath -force -axis 3 $dwi_den_deg_preproc_warp_unb_b0s mean $dwi_den_deg_preproc_warp_unb_meanb0
    
    ##c. Convert mean b0 to NII.GZ
    dwi_den_deg_preproc_warp_unb_meanb0_NIIGZ = preprocdir/(prefix + "_dwi_den_deg_preproc_warp_unb_meanb0.nii.gz")
    !mrconvert -force $dwi_den_deg_preproc_warp_unb_meanb0 $dwi_den_deg_preproc_warp_unb_meanb0_NIIGZ
    
    ##d. Create mask
    dwi_den_deg_preproc_warp_unb_meanb0maskroot = preprocdir/(prefix + "_dwi_den_deg_preproc_warp_unb_meanb0")
    !bet2 $dwi_den_deg_preproc_warp_unb_meanb0_NIIGZ $dwi_den_deg_preproc_warp_unb_meanb0maskroot -m
    
    ##e. Convert mask back to MIF
    dwi_den_deg_preproc_warp_unb_meanb0mask_NIIGZ = preprocdir/(prefix + "_dwi_den_deg_preproc_warp_unb_meanb0_mask.nii.gz")
    dwi_den_deg_preproc_warp_unb_meanb0mask = preprocdir/(prefix + "_dwi_den_deg_preproc_warp_unb_meanb0_mask.mif")
    !mrconvert -force $dwi_den_deg_preproc_warp_unb_meanb0mask_NIIGZ $dwi_den_deg_preproc_warp_unb_meanb0mask
    
    ##f. Upsample mask
    dwi_den_deg_preproc_warp_unb_meanb0mask_ups = preprocdir/(prefix + "_dwi_den_deg_preproc_warp_unb_meanb0_mask_ups.mif")
    !mrgrid -force $dwi_den_deg_preproc_warp_unb_meanb0mask regrid $dwi_den_deg_preproc_warp_unb_meanb0mask_ups -template $dwi_den_deg_preproc_warp_unb_ups -interp linear -datatype bit
    
    ##g. Filter mask
    dwi_den_deg_preproc_warp_unb_meanb0mask_ups_filt = preprocdir/(prefix + "_dwi_den_deg_preproc_warp_unb_meanb0_mask_ups_filt.mif")
    !maskfilter -force $dwi_den_deg_preproc_warp_unb_meanb0mask_ups median $dwi_den_deg_preproc_warp_unb_meanb0mask_ups_filt
    
    ########################################
    #----Copying Preprocessed DTI Files----#
    ########################################
    
    #1. Make processing directory for subject and output path variables
    procsubj_dir = Path(procsubj_dir)
    procsubj_dir.mkdir(parents=True, exist_ok=True)
    dwiproc_out = procsubj_dir/(prefix + "_ppd.mif")
    dwimaskproc_out = procsubj_dir/(prefix + "_mask_ppd.mif")
    
    #2. Copy preprocessed DTI volumes
    shutil.copy(dwi_den_deg_preproc_warp_unb_ups, dwiproc_out)
        
    #3. Copy preprocessed mask
    shutil.copy(dwi_den_deg_preproc_warp_unb_meanb0mask_ups_filt, dwimaskproc_out)

In [2]:
bidsmaster_dir = Path("/Volumes/Vol6/YouthPTSD/BIDS_master/")
bidspreproc_dir = Path("/scratch/jdrussell3/bidspreproc/")
bidsproc_dir = Path("/scratch/jdrussell3/bidsproc")
eddy_options = "'--verbose --slm=linear --repol --cnr_maps --residuals'"
dwelltime = 0.000568
totalreadouttime = 0.14484

subjlist1of3 = ['sub-001', 'sub-002', 'sub-003', 'sub-004', 'sub-005', 'sub-006', \
                'sub-007', 'sub-008', 'sub-009', 'sub-011', 'sub-012', 'sub-013', \
                'sub-014', 'sub-019', 'sub-020', 'sub-021', 'sub-023', 'sub-024', \
                'sub-025', 'sub-026', 'sub-028', 'sub-029', 'sub-031', 'sub-035', \
                'sub-036', 'sub-037', 'sub-041', 'sub-042', 'sub-043', 'sub-044']
subjlist2of3 = ['sub-045', 'sub-050', 'sub-056', 'sub-057', 'sub-058', 'sub-059', \
                'sub-060', 'sub-061', 'sub-062', 'sub-064', 'sub-065', 'sub-068', \
                'sub-070', 'sub-071', 'sub-073', 'sub-075', 'sub-076', 'sub-078', \
                'sub-079', 'sub-081', 'sub-082', 'sub-084', 'sub-085', 'sub-086', \
                'sub-087', 'sub-089', 'sub-090', 'sub-091', 'sub-092', 'sub-093', \
                'sub-094', 'sub-097', 'sub-099', 'sub-100', 'sub-101', 'sub-104'] 
subjlist3of3 = ['sub-106', 'sub-107', 'sub-108', 'sub-111', 'sub-112', 'sub-114', \
                'sub-117', 'sub-118', 'sub-122', 'sub-124', 'sub-125', 'sub-127', \
                'sub-128', 'sub-129', 'sub-131', 'sub-132', 'sub-133', 'sub-134', \
                'sub-135', 'sub-138', 'sub-139', 'sub-140', 'sub-141', 'sub-142', \
                'sub-145', 'sub-146', 'sub-147', 'sub-148', 'sub-149', 'sub-151', \
                'sub-153', 'sub-154', 'sub-155', 'sub-156', 'sub-157']

subjdirs = (subjdir for subjdir in bidsmaster_dir.iterdir() if subjdir.is_dir() \
            and subjdir.name in subjlist2of3)
   
for subjdir in sorted(subjdirs):
    sesdirs = (sesdir for sesdir in subjdir.iterdir() if sesdir.is_dir())
    for sesdir in sorted(sesdirs):
        dwidir = sesdir/'dwi'
        fmapdir = sesdir/'fmap'
        if dwidir.exists() and fmapdir.exists():
            subjroot = "_".join([subjdir.name, sesdir.name])
            sourcedwi = dwidir/(subjroot + "_acq-AxDTIASSET_dwi.nii")
            sourcebvec = dwidir/(subjroot + "_acq-AxDTIASSET_dwi.bvec")
            sourcebval = dwidir/(subjroot + "_acq-AxDTIASSET_dwi.bval")
            sourcefmap_rads = fmapdir/(subjroot + "_acq-RealFieldmapDTIrads_fmap.nii")
            sourcefmap_mag = fmapdir/(subjroot + "_acq-FieldmapDTI_magnitude1.nii")
            preprocsubj_dir = Path(bidspreproc_dir, subjdir.name, sesdir.name, 'dti')
            procsubj_dir = Path(bidsproc_dir, subjdir.name, sesdir.name, 'dti')
            eddy_options = eddy_options
            dwelltime = dwelltime
            totalreadouttime = totalreadouttime
            try:
                dwi_corr(sourcedwi, sourcebvec, sourcebval, sourcefmap_rads, sourcefmap_mag,\
                         preprocsubj_dir, procsubj_dir, eddy_options, dwelltime, totalreadouttime)
            except Exception as e:
                print(e)
                errfile = Path(bidspreproc_dir / 'error_log.txt')
                with open(errfile, 'a+') as errorfile:
                    errorfile.write(str('Error processing ' + subjdir.name + ', ' + \
                                        sesdir.name + ':\n' + str(e) + '\n'))
                next
%%capture cap          
with open('output.txt', 'w') as f:
    f.write(cap.stdout)

sub-045_ses-01
mrconvert: [WARNING] existing output files will be overwritten
mrconvert: [100%] copying from "/scratch/j...inal/sub-045_ses-01_dwi.nii" to "/scratch/j...ssed/sub-045_ses-01_dwi.mif"
dwidenoise: [WARNING] existing output files will be overwritten
dwidenoise: [100%] preloading data for "/scratch/jdrussell3/bidspreproc/sub-045/ses-01/dti/preprocessed/sub-045_ses-01_dwi.mif"
dwidenoise: [ 71%] running MP-PCA denoising... mrdegibbs: [WARNING] existing output files will be overwritten
mrdegibbs: [  2%] performing Gibbs ringing removal... ^C

mrdegibbs: [SYSTEM FATAL CODE: SIGINT (2)] Program manually interrupted by terminal
^C
Traceback (most recent call last):
  File "/Users/jdrussell3/apps/mrtrix3/bin/dwifslpreproc", line 1146, in <module>
    mrtrix3.execute() #pylint: disable=no-member
  File "/Volumes/Users/jdrussell3/apps/mrtrix3/lib/mrtrix3/__init__.py", line 88, in execute
    from . import app #pylint: disable=import-outside-toplevel
  File "/Volumes/Users/jdrussell3

KeyboardInterrupt: 

mrconvert: [WARNING] existing output files will be overwritten
mrconvert: [100%] copying from "/scratch/j...inal/sub-045_ses-02_dwi.nii" to "/scratch/j...ssed/sub-045_ses-02_dwi.mif"
dwidenoise: [WARNING] existing output files will be overwritten
dwidenoise: [100%] preloading data for "/scratch/jdrussell3/bidspreproc/sub-045/ses-02/dti/preprocessed/sub-045_ses-02_dwi.mif"
dwidenoise: [ 18%] running MP-PCA denoising... 